# Model Comparison

In [19]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

In [2]:
#set golbal path
global_path=os.path.abspath('.')
#load the needed file
df_all = pd.read_csv(global_path+'/data/inputdata.csv')
df_train = pd.read_csv(global_path+'/data/train_users_2.csv')
df_test = pd.read_csv(global_path+'/data/test_users.csv')

In [4]:
#get the needed parameter for model training and testing
label = df_train['country_destination']
#get the id of test
id_test = df_test['id']

# Build the Traing set ,Label and Test set

In [12]:
######Computing X, y and X_test ################
piv_train = len(label) #Marker to split df_all into train + test
vals = df_all.values
le = LabelEncoder()

X = vals[:piv_train]
y = le.fit_transform(label.values)
X_test = vals[piv_train:]

## Xgboost Model

### 0.87822 accuracy for Xgboost algorithm

In [ ]:
#Classifier
xgb = XGBClassifier(max_depth=6, learning_rate=0.3, n_estimators=25,
                    objective='multi:softprob', subsample=0.5, colsample_bytree=0.5, seed=0)                  
bst = xgb.fit(X, y)
y_pred = bst.predict_proba(X_test)  

#Taking the 5 classes with highest probabilities
ids = []  #list of ids
cts = []  #list of countries
for i in range(len(id_test)):
    idx = id_test[i]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_pred[i])[::-1])[:5].tolist()
#generate submit data frmae
sub_xgb = pd.DataFrame(np.column_stack((ids, cts)), columns=['id', 'country'])
sub_xgb.to_csv('sub_xgb.csv',index=False)

## K-NN Model

### 0.80598 accuracy for K-NN algorithm

In [13]:
neigh = KNeighborsClassifier(n_neighbors=3)
Kmodel = neigh.fit(X, y) 
y_pred_knn = Kmodel.predict_proba(X_test)

#Taking the 5 classes with highest probabilities
ids = []  #list of ids
cts = []  #list of countries
for i in range(len(id_test)):
    idx = id_test[i]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_pred_knn[i])[::-1])[:5].tolist()
#generate submit data frmae    
sub_knn = pd.DataFrame(np.column_stack((ids, cts)), columns=['id', 'country'])
#save submit file
sub.to_csv('sub_knn.csv',index=False)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

## SVM Model

### 0.84679 accuracy for SVM algorithm

In [21]:
clf = svm.SVC()
svm_model = clf.fit(X, y)  
y_pred_svm = svm_model.predict_proba(X_test)

#Taking the 5 classes with highest probabilities
ids = []  #list of ids
cts = []  #list of countries
for i in range(len(id_test)):
    idx = id_test[i]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_pred_svm[i])[::-1])[:5].tolist()
#generate submit data frmae    
sub_svm = pd.DataFrame(np.column_stack((ids, cts)), columns=['id', 'country'])
#save submit file
sub.to_csv('sub_svm.csv',index=False)